# Pseudo-code discussion towards v2

Authors: **Gorka Zamora-López** and **Matthieu Gilson**

---------------------

The goal of this notebook is to try, propose and discuss how we would like that typical workflows look like in the object-oriented version of *SiReNetA* (v2). The notebook is based on the workflow described in the first tutorial [notebook](https://github.com/mb-BCA/SiReNetA_Tutorials/blob/master/Tutorial_Notebooks/1_GettingStarted.ipynb).

See also further discussions in the [TODO_Future.md](https://github.com/mb-BCA/SiReNetA/blob/v1_dev/TODO_Future.md) file.

----------------------

In [ ]:
# Python standard & third-party library imports
from timeit import default_timer as timer

import matplotlib.pyplot as plt
import numpy as np

import sireneta as sna
print( 'SiReNetA:', sna.__version__ )

In [ ]:
# Define plotting options to control visualization
%matplotlib inline

# Load the options from a local file
from plot_specs import *

We consider the network determined by the following binary matrix.

In [ ]:
# Load the sample graph to study
net = np.loadtxt('../Data/Testnet_N8.txt', dtype=int)
# Number of nodes
N = len(net)

Convergence of the leaky-cascade model depends on the leakage time-constant, when $\tau \leq 1 \,/\, \lambda_{max}$. Find the largest eigenvalue of connectivity *A* and the critical $\tau_{max}$.

In [ ]:
# Find the largest eigenvalue of the connectivity matrix A
evs = np.linalg.eigvals(net)
evmax = evs.real.max()
# Calculate the largest possible tau
taumax = 1.0 / evmax

print( f'Spectral radius:\t{evmax:2.5f}' )
print( f'Largest possible tau:\t{taumax:2.5f}' )

<br>

## 1. Initialize the main container object (instance)

Object `Rmats` must be initialized with two mandatory parameters:

* `con` : The connectivity matrix.
* `canmod` : identifier for the canonical model. Probably a string.

These two parameters shall be **_unmutable_**. They cannot be changed after object creation. `con` will be a connectivity matrix, either loaded from data or generated with some graph library.


> WARNING ! `con` is likely to be generated by a graph analysis software, or loaded from data as 

**Gorka**: I don't think we need to initialize `Rmats` with more parameters.

TODO : Probably, find a better name for the creation function `PairWiseResp()`.

In [ ]:
# Rmat must be initialized for a connectivity matrix and a canonical model.
# These are unmutable and cannot be changed. Ever !
Rmats = sna.PairWiseResp(con=net.T, canmod='LeakyCascade')


After this, `Rmats` should also *know* whether `con` is directed or undirected and the number of nodes *N*. E.g.:

``` python
>>> Rmats.N
8
>>> Rmats.directed
False

```

## 2. Prepare the conditions and compute the pair-wise responses

For now, I see two ways in which the needed metadata could be inputed into the `Rmats` object and compute the pair-wise responses.

----------------------

### Option 1: Feed `Rmats` with parameters and then call the calculation of $\mathcal{R}_{ij}(t)$.
In this option, the `Rmats` object is first populated with all the parameters needed for the "simulation" (feed the necessary attributes) and then the pair-wise responses are calculated.

1. Feed the different parameters (attributes) to `Rmats`, e.g.:
    - `Rmats.max = 10`
    - `Rmats.tau = 0.8 * taumax`
    - ...
2. Calculate the pairwise responses `Rmats.Calc_Resp()`

I think this is quite a traditional way, although not my favourite choice (Gorka) because it is not flexible in case the "simulation" needs to be run again, e.g. because the `tmax` was not long enough and responses didn't converge enough. See *Option 2* and the discussion below.

In [ ]:
# Define the "simulation" parameters
# Set the temporal resolution
Rmats.tmax = 10
Rmats.timestep = 0.01

# Set the leakage time-constants τ, proportional to taumax
# NOTE: Actually, `taumax` should also be an attribute of Rmats, right?
Rmats.tau = 0.8 * taumax

# Define the stimulation amplitude to every node
# Example of stimulus on one node
Rmats.S0 = np.zeros(N)
Rmats.S0[0] = 1.0

# Example of all nodes receive same stimulus
Rmats.S0 = 1.0

In [ ]:
# Calculate temporal evolution of the pair-wise responses R(t)
Rmats.Calc_Resp(case='regressed') 

In this option, all the parameters needed for the "simulation" have been already stored as attributes of `Rmats`, before calling the `Calc_Resp()` function.
These are: `S0`, `tmax`, `timestep` and `tau`. Parameter `case` is rather optional and only available in two of the five canonical models. `case` could be stored before calling the function, or added as an attribute to `Rmats` at function call.


----------------------

### Option 2: Define the parameters externally, and then call the calculation of $\mathcal{R}_{ij}(t)$.
In this option, the "simulation" parameters are defined in a more traditional way (non-OO). The parameters are defined and explicitly entered into the `Calc_Resp()` function. It is only when `Calc_Resp()` is called, that the attributes of `Rmats` are populated or updated.

1. Feed the different parameters (attributes) to `Rmats`, e.g.:
    - `tfinal = 10 10`
    - `dt = 0.01` 
    - `tau = 0.8 * taumax`
    - `stimvec = np.ones(N)` 
2. Calculate the pairwise responses `Rmats.Calc_Resp(S0=stimvec, tmax=tfinal, timestep=dt, case='regressed')`.

I think this is quite a traditional way, although not my favourite choice (Gorka) because it is not flexible in case the "simulation" needs to be run again, e.g. because the `tmax` was not long enough and responses didn't converge enough. See *Option 2* and the discussion below.

In [ ]:
# Define the "simulation" parameters
# Set the temporal resolution
tfinal = 10
dt = 0.01

# Set the leakage time-constants τ, proportional to taumax
tau = 0.8 * taumax

# Define the stimulation amplitude to every node
stimvec = 1.0


In [ ]:
# Calculate temporal evolution of the pair-wise responses R(t)
## Here, all necessary "simulation" parameters (`S0`, `tmax`, `timestep` and `tau`)
## shall be manually entered. `case`is an optional parameter.
Rmats.calc_Resp(S0=stim, tau=tau, tmax=tfinal, timestep=dt, case='regressed')

In this option, the crucial parameters needed for the "simulation" (`S0`, `tmax`, `timestep` and `tau`) need to be entered to `Calc_Resp()` explicitely. Then, when the function is called, the function fills or updates those attributes into `Rmats` object. 

Q : Actually, is there a way in Python to prevent that a user externally updates attributes `S0`, `tmax`, `timestep` and `tau`, and that these can **only** be updated from inside `Calc_Resp()` function ? That would make things pretty safe.

----------------------

**Gorka** : Why do I prefer this option? I think it is safer that the crucial metadata will not be changed accidentally by the user or anything else. For example, attribures `tmax` and `timestep` are not directly modified by the user, but are only updated when calling the `Calc_Resp()` function. 

Imagine we compute the responses for tmax = 10 and timestep = 0.1. We plot the result and realize that this temporal scale is too large. We want to recalculate the responses (without having to initialize a new instance) for tmax = 1.0 and timestep=0.001.
In Option-1 that would mean to edit manually the attributes `Rmats.tmax=1.0` and `Rmats.timestep=0.001`. Then, we have to deliberatelly call again `Rmats.Calc_Resp()` in order to update the results. But, what if the user, after changing `tmax` and `timestep` attributes, forget to call `Calc_Resp()` ? In that case, the metadata of `Rmats` does no longer match the one used to previously compute the responses, whose result is still saved into an array of shape (10//0.1+1,N,N), instead of the (1.0//0.001+1,N,N) now expected. This situation might not be very usual when scripting but … I can easily see things like this happening when working on Jupyter Notebooks.

In Option-2, the only time that attributes are allowed to change are at the call of `Calc_Resp()` function, and only this function has the permission to change them.


### Output of `Calc_Resp()` function

The main result of `Calc_Resp()` function (method) is to compute the response matrices over time. This is stored into a 3D numpy array `data` of shape (tmax//dt+1, N, N). The "+1" is because the first time step is *t=0*. All subsequent results are obtained out of `Rmats.data`, either slicing the 3D array projecting along different axes, or by estimating metrics out of it.

### Plot some results

For example, visualize the response matrices at selected time points.

In [ ]:
# Visualise the pair-wise response matrices at times t = 0.1, 0.3, 0.5, 1.0, 2.0, 3.0
maxresp = Rmats.max()

tidxlist = [10,20,50,100,200,500]
plt.figure(figsize=[12,6])
for i, tidx in enumerate(tidxlist):
    t = tpoints[tidx]
    plt.subplot(2,3,i+1)
    plt.title( f'$\mathcal{{R}}(t)$ matrix at t={t:1.1f}' )
    plt.imshow(Rmats.data[tidx], cmap=new_Reds)
    
    plt.clim(0,maxresp)
    plt.colorbar()
    plt.xticks(np.arange(N), np.arange(N)+1)
    plt.yticks(np.arange(N), np.arange(N)+1)
    plt.xlabel( 'source node' )
    plt.ylabel( 'target node' )

plt.tight_layout()

In [ ]:
# Plot a few response curves. 
# The responses of i = 2, ..., N to the stimulus at j = 1.

plt.figure(figsize=(6.4,3))
for i in range(1,N):
    plt.plot(Rmats.tpoints, Rmats.data[:,i,0], label=f'1 $\\to$ {i+1}')
plt.xlabel( 'Time (a.u.)' )
plt.ylabel( 'Pair-wise Response' )
plt.legend()

plt.tight_layout()

<br>

## 3. Slicing and projecting `Rmats.data`. Extracting metrics.

I have a major doubt here. After the tensor containing $\mathcal{R}_{ij}(t)$, the other two time-series "arrays" are the global response $R(t)$ and the pair-wise responses $r_i(t)$. `Rglob` is a 1D ndarray of length `nsteps` and `rnodes` is a 2D ndarray of shape (`nsteps`,`N`).

The question is whether `Rglob` and `rnodes` 
1. should be themselves two independent objects which inherit the temporal attributes from `Rmats`, and the methods (e.g., `Time2Peak()` and `AreaUnderCurve()`) or 
2. they should be directly considered attributes of `Rmats`. Q: Can attributes be objects themselves, that share selected attributes?  

Personally, I would have a preference for the first case but, for mainly because that would allow me to keep the the workflows closer to what I am used to. But I ignore the details of what would be easier and more consistent to code internally.

If we consider them **as independent objects**, we would for example do as follows:

In [ ]:
# Compute the global network response
Rglob = sna.GlobalResponse(Rmats)



As observed, the largest (*and fastest* !) responses correspond to *i = 2, 3* which are directly connected to *j = 1*, followed by *4* and *6* which are the nodes with more connections in the network.

### Step 4 …

… is finally to _**extract network metrics out of**_ $\mathcal{R}\_{ij}(t)$, in the form of spatio-temporal properties of the response propagation. There are many such metrics that could be derived. Here we only estimate two for illustration (see [Calculating Response to Stimulus and Metrics](2_Basics_StimRespMetrics.ipynb) for further metrics):


- Time-to-peak distance $D^{ttp}_{ij}$, which analogous to the graph distance, and …
- the total node responses, $\bar{r}_i$, which is reminiscent of the degree or centrality of a node.



The pair-wise responses $\mathcal{R}_{i \gets 1}(t)$ plotted above showed that every pair-wise interaction peaks at different times $t^*_{i1}$. This pair-wise time-to-peak can be employed to redefine the distance between nodes in a network ($D^{ttp}_{ij} = t^*_{ij}$) and thus  generalize the usual path distance in graphs. See [Network Distance](5_UseCase_NetDist.ipynb) and [Zamora-López & Gilson, Chaos (2024)](https://doi.org/10.1063/5.0202241) for further details.

In the following we compute and visualize this distance matrix:

In [ ]:
# Compute the time-to-peak distance 
ttpdist = sna.Time2Peak(Rmats, timestep=dt)

plt.figure(figsize=(10,3))
# Plot the adjacency matrix
plt.subplot(1,2,1)
plt.title( 'Adjacency matrix' )
plt.imshow(net, cmap='gray_r')
plt.colorbar()
plt.xlabel( 'Node index' )
plt.ylabel( 'Node index' )

# Plot the time-2-peak distance matrix
plt.subplot(1,2,2)
plt.title( 'Time-to-peak distance' )
plt.imshow(ttpdist)
plt.colorbar()
plt.xlabel( 'Node index' )
plt.ylabel( 'Node index' )

plt.tight_layout()

As expected from the $\mathcal{R}_{i \gets 1}(t)$ curves above, the nodes that are directly connected peak earlier to the stimuli and are thus "closer".

Finally, we compute the total node responses $\bar{r}_i$. For that, we first compute the (summed) response $r_i(t)$ of *i* to the stimuli in all other *j*. Then $\bar{r}_i$ is the accumulated response of *i* over time (integral or area-under-the-curve of $r_i(t)$).

In [ ]:
# Compute and visualise the node responses
r_nodes = sna.NodeResponses(Rmats)[0]

# Visualize the node responses over time
plt.figure(figsize=(6.4,3.8))
for i in range(N):
    plt.plot(tpoints, r_nodes[:,i], label=f'Node {i+1}')
plt.plot(tpoints, Rglob / N, label='Global', c='gray', ls='--', lw=4)
plt.xlabel( 'Time (a.u.)' )
plt.ylabel( 'Node Response' )
plt.legend()

plt.tight_layout()

Nodes *i = 4* and *6* are those with largest responses, since they are the most connected. On the contrary, *i = 5* and *7* have only one connection and thus display the smallest responses.

We now reduce this into one value per node, the total node response $\bar{r}_i$ and compare to their degrees $k_i$.

In [ ]:
# Compute the total node responses
totr_nodes = sna.AreaUnderCurve(r_nodes, dt)

# Visualize the total node responses
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
for i in range(N):
    plt.bar((i+1), totr_nodes[i])
plt.xlabel( 'Node index' )
plt.ylabel( 'Total node resp.  $\\bar{r}_i$' );

# Visualize their relation with the degree
deg = net.sum(axis=0)

plt.subplot(1,2,2)
for i in range(N):
    plt.scatter(deg[i], totr_nodes[i])
plt.xlabel( 'Degree of node, $k_i$' )
plt.ylabel( 'Total node resp.  $\\bar{r}_i$' )

plt.tight_layout()

Last, we can explore _**the relation between the centrality of a node and their total response**_. 

In classical graph theory, the degree of a node is a first-order measure of centrality. The  closeness centrality $c_i$ is defined as the sum of (geodesic) distances from node *i* to the others: $c_i = \sum_{k=1}^N D_{ik}$. Both degree and closeness centrality are typically related via a negative correlation: the more connected a node (like a hub), the closer it is on average to other nodes via network paths.

Analogously, closeness centrality can be defined from the time-to-peak distance matrix as $c_i = \sum_{k=1}^N  D^{ttp}_{ik}$. Below, we compare this closeness centrality with the degree and the total node responses.

In [ ]:
# Compute closeness centrality from time-to-peak distances
c_nodes = ttpdist.sum(axis=1)

plt.figure(figsize=(10,3))
# Compare centrality with degree
plt.subplot(1,2,1)
for i in range(N):
    plt.scatter(deg[i],c_nodes[i])
plt.xlabel( 'Degree of Node, $k_i$' )
plt.ylabel( 'Closeness cent.' )

# Compare centrality with total node responses
plt.subplot(1,2,2)
for i in range(N):
    plt.scatter(totr_nodes[i],c_nodes[i])
plt.xlabel( 'Total node resp.  $\\bar{r}_i$' )
plt.ylabel( 'Closeness cent.')

plt.tight_layout()

We see the expected negative correlations in both figures, the second one following because the degree and the total response are positively correlated. This shows that, for this example small network, nodes that have high degree (a local property) are also those "important" for the centrality (a global property that takes the whole network into account).

Note that the claimed correspondance between geodesic distance and time-to-peak is valid **because the considered network is binary = unweighted**. However, the dynamical measures here proposed are also straight-forwardly applicable to weighted networks, unlike many graph metrics: for instance, a weight could correspond to a cost (like with the geodesic distance) or an efficacy to propagate the signal. Care must thus be taken when applying and interpreting spatio-temporal measures applied to a particular network. See [Weighted Networks](6_Basics_WeightedNets.ipynb) for examples of the applications to weighted cases.